1. Дан массив вида [a,b,c,d,e,a,a,b,c]

а) Преобразуйте его в словарь вида
{
   a: [0,5,6],
   b: [1,7],
   ...
   
   e: [5]
}

б) Сделайте это в одну строчку кода (разрешается с меньшей эффективностью)


In [ ]:
matrix = ['a','b','c','d','e','a','a','b','c']
{x:[y for y in range(len(matrix)) if x==matrix[y]] for x in matrix}


2. Метрика Жаккара (Jaccard similarity) - способ измерения сходства между множествами. Вычисляется как |AnB|/|AuB|, т.е. размер пересечения поделить на размер объединения множеств.

Реализуйте вычисление метрики не используя методов intersection и union типа set и не создавая промежуточных множеств



In [ ]:
A=[1,2,3,4,5,6,7,8,9]
B=[1,2,3,42]
len([x for x in A if x in B])/(len(A)+len(B))


4. Дан json-файл вида:
[
{
      "item": "ITEM1",
      "sales_by_country": {
            "Russia": {
	    “2010”: 23,
	   “2011”: 40
	    ...
	   “2019”: 300
	},
	“Ukraine”: {
	     ….
	},
	….
     } 
},
{
      "item": "ITEM2",
      "sales_by_country": {
            "Russia": {
	    “2010”: 43,
	    ...
	   “2019”: 30
	},
	“Ukraine”: {
	     ….
	},
	….
     } 
},
...
] 

Преобразуйте данные в CSV-файл вида:

item, country, year, sales
ITEM1, Russia, 2010, 23
ITEM1, Russia, 2011, 40 
…
ITEM2, Russia, 2010, 40 
....


In [ ]:
import csv
import json

with open("../input/saless/sales.json", "r") as read_file: 
    df = json.load(read_file)
    
header = ['item', 'country', 'year', 'sales']
df_csv = [header]
for item in df:
    cur_item = item['item']
    #print(item['item']) #rows items
    for country in item['sales_by_country'].items():
        cur_country = country[0]
        #print(country[0]) #country rows
        for year in country[1].items():
            cur_year = str(year[0])
            cur_sales = str(year[1])
            df_csv.append([cur_item,cur_country, cur_year, cur_sales])
df_csv
            
with open('sales.csv', 'w') as write_file:
    writer = csv.writer(write_file)
    for row in df_csv:
        writer.writerow(row)
        
with open('sales.csv') as read_file:
    print(read_file.read())

5. Используя API центрального банка (https://www.cbr.ru/development/) соберите данные по курсам валют (по отношению к рублю) c 1 марта 2020 по 1 июля 2020
Необходимые поля:
Дата
Доллар США
Евро
Индийская Рупия
Украинская гривна



In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv
from time import sleep

url = "http://www.cbr.ru/scripts/XML_dynamic.asp"
#динамика курса валюты за временной промежуток

def get_data():
    valutes = {
        'americ_dollar': 'R01235',
        'euro':'R01239',
        'indian_rupee':'R01270',
        'ukr_hryvna':'R01720'
         }
    #Dollar,Euro,Rupee,Hryvna
    for country,valute in valutes.items():
        temp = url + F'?date_req1=01/03/2020&date_req2=01/07/2020&VAL_NM_RQ={valute}'
        webFile = requests.get(temp)
        with open(F"{country}.xml", "w") as localFile:
            localFile.write(webFile.text)
        webFile.close()

    data = read_xml(valutes.keys())
    csv_lazy_writer(valutes.keys(), data)

def read_xml(countries):
    temp_data = []
    for ctr in countries:
        tree = ET.parse(F"{ctr}.xml")
        root = tree.getroot()
        for first_child in root:
            raw_data = {}
            
            raw_data["date"] = first_child.attrib['Date']
            raw_data['ctr'] = ctr
            for second_child in first_child:
                raw_data[second_child.tag] = second_child.text
            temp_data.append(raw_data)
    return temp_data

def csv_lazy_writer(countries,data):
    with open("all_countries.csv", "w", newline='') as out_file:
        writer = csv.DictWriter(out_file, delimiter=',', fieldnames=['date', 'ctr', 'Nominal', 'Value'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)


In [ ]:
get_data()

6*. Дан граф друзей из соцсети VK (https://www.dropbox.com/s/bjoji6y47vwr90d/arcs_refined.txt?dl=0):
Необходимо:
1) Определить число уникальных пользователей
2) Для каждого id пользователя определить число его друзей из данного графа. Вывести топ 15. Учитывать неориентированность графа.
3) Для всех возможных пар пользователей (декартова произведения):
Для длины L 1..6 определить пропорцию пар пользователей, для которой кратчайший путь между пользователями в паре составляет L. 
Определить пропорцию, для которой этот путь больше 6.
Убедиться, что полученные пропорции суммируются в 1.


In [ ]:
#агрегация данных
with open("../input/friends/arcs_refined.txt", "r") as file:
    users = {}
    for line in file:
        line = line.split()
        if line[0] not in users.keys():
            users[line[0]] = set()
        if line[1] not in users.keys():
            users[line[1]] = set()
        users[line[0]].add(line[1])
        users[line[1]].add(line[0])
users

In [ ]:
#Определить число уникальных пользователей
len(users.keys())


In [ ]:
#Для каждого id пользователя определить число его друзей из данного графа. 
#Вывести топ 15. Учитывать неориентированность графа.
def combsort(alist):
    alen = len(alist)
    gap = (alen * 10 // 13) if alen > 1 else 0
    while gap:
        if 8 < gap < 11:    ## variant "comb-11"
            gap = 11
        swapped = False
        for i in range(alen - gap):
            if alist[i + gap] > alist[i]:
                alist[i], alist[i + gap] = alist[i + gap], alist[i]
                swapped = True
        gap = (gap * 10 // 13) or swapped
    return(alist)
        
friends_count = []
for vk_id,friends_set in users.items():
    friends_count.append([len(friends_set),vk_id])
friends_count = combsort(friends_count)
friends_count[:15] #Самым дружелюбным оказался зам. председателя профкома 

In [ ]:
#3) Для всех возможных пар пользователей (декартова произведения): 
#Для длины L 1..6 определить пропорцию пар пользователей, 
#для которой кратчайший путь между пользователями в паре составляет L. 
#Определить пропорцию, для которой этот путь больше 6. 
#Убедиться, что полученные пропорции суммируются в 1.

#Инициализация списка расстояний
dist = {}
vertex = list(users.keys())
for id1 in vertex:
    for id2 in vertex[vertex.index(id1)+1:]:
        temp = 1 if id2 in users[id1] else len(vertex)
        dist[(id1,id2)] = temp
        
f = open("test.txt", "w")
for key,val in dist.items():
        f.write('{}:{}\n'.format(key,val))
f.close()

In [ ]:
#Алгоритм Флойда-Уоршелла для нахождения расстояния между всеми парами
for k in vertex:
    for u in vertex:
        if (u!=k): 
            for v in vertex:
                if (v!=u):
                    if dist.get((u,v)) and dist.get((u,k)) and dist.get((k,v)):
                        dist[(u,v)] = min(dist[(u,v)], dist[(u,k)] + dist[(k,v)])
                    #else:
                        #dist[(v,u)] = min(dist[(v,u)], dist[(v,k)] + dist[(k,u)])

f = open("test2.txt", "w")
for key,val in dist.items():
        f.write('{}:{}\n'.format(key,val))
f.close()

In [ ]:
#Нахождение пропорций
def proportion(L, dist):
    count = 0
    for value in dist.values():
        if value == L: count += 1
    result = count / len(dist.keys())
    return(result)

L = []
for i in range(1,7):
    L.append(proportion(i, dist))
summary = sum(L)
L.append(1 - summary)
L